In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-midterm-p1/train.csv
/kaggle/input/2022-ai-midterm-p1/test.csv
/kaggle/input/2022-ai-midterm-p1/submit_sample.csv


In [2]:
#랜덤시드 고정
import random
import torch

seed = 1
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
#데이터 로드
sample = pd.read_csv('/kaggle/input/2022-ai-midterm-p1/submit_sample.csv')
train = pd.read_csv('/kaggle/input/2022-ai-midterm-p1/train.csv')
test = pd.read_csv('/kaggle/input/2022-ai-midterm-p1/test.csv')

In [4]:
#gpu 사용가능하면 사용하는 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
#데이터 정보 확인
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1437 entries, 0 to 1436
Data columns (total 66 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   index   1437 non-null   int64
 1   0       1437 non-null   int64
 2   1       1437 non-null   int64
 3   2       1437 non-null   int64
 4   3       1437 non-null   int64
 5   4       1437 non-null   int64
 6   5       1437 non-null   int64
 7   6       1437 non-null   int64
 8   7       1437 non-null   int64
 9   8       1437 non-null   int64
 10  9       1437 non-null   int64
 11  10      1437 non-null   int64
 12  11      1437 non-null   int64
 13  12      1437 non-null   int64
 14  13      1437 non-null   int64
 15  14      1437 non-null   int64
 16  15      1437 non-null   int64
 17  16      1437 non-null   int64
 18  17      1437 non-null   int64
 19  18      1437 non-null   int64
 20  19      1437 non-null   int64
 21  20      1437 non-null   int64
 22  21      1437 non-null   int64
 23  22      1437 

In [6]:
train

,index,0,1,2,3,4,5,6,7,8,...,55,56,57,58,59,60,61,62,63,label
0,0,0,0,5,13,9,1,0,0,0,...,0,0,0,6,13,10,0,0,0,0
1,1,0,0,0,12,13,5,0,0,0,...,0,0,0,0,11,16,10,0,0,1
2,2,0,0,0,4,15,12,0,0,0,...,0,0,0,0,3,11,16,9,0,2
3,3,0,0,7,15,13,1,0,0,0,...,0,0,0,7,13,13,9,0,0,3
4,4,0,0,0,1,11,0,0,0,0,...,0,0,0,0,2,16,4,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1432,1792,0,0,4,10,13,6,0,0,0,...,0,0,0,2,14,15,9,0,0,9
1433,1793,0,0,6,16,13,11,1,0,0,...,0,0,0,6,16,14,6,0,0,0
1434,1794,0,0,1,11,15,1,0,0,0,...,0,0,0,2,9,13,6,0,0,8
1435,1795,0,0,2,10,7,0,0,0,0,...,0,0,0,5,12,16,12,0,0,9


In [7]:
#데이터 전처리
xtrain = train.drop(['index','label'],axis=1)
ytrain = train['label']
xtest = test.drop(['index'],axis=1)

In [8]:
X_train = torch.FloatTensor(np.array(xtrain)).to(device)
y_train = torch.LongTensor(np.array(ytrain)).to(device)
X_test = torch.FloatTensor(np.array(xtest)).to(device)

In [9]:
#모델 빌드를 위한 설정값 확인
print(X_train.shape)
print(y_train.shape)
print(len(y_train.unique()))

torch.Size([1437, 64])
torch.Size([1437])
10


In [10]:
#모델 빌드
import torch.nn as nn

class DNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(64, 512, bias=True)
        self.linear2 = nn.Linear(512, 1024, bias=True)
        self.linear3 = nn.Linear(1024, 1024, bias=True)
        self.linear4 = nn.Linear(1024, 10, bias=True)
        
        self.active = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        
        nn.init.xavier_normal_(self.linear1.weight)
        nn.init.xavier_normal_(self.linear2.weight)
        nn.init.xavier_normal_(self.linear3.weight)
        nn.init.xavier_normal_(self.linear4.weight)
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.active(x)
        x = self.dropout(x)
        x = self.linear2(x)
        x = self.active(x)
        x = self.dropout(x)
        x = self.linear3(x)
        x = self.active(x)
        x = self.dropout(x)
        x = self.linear4(x)
        
        return x

In [11]:
#모델 정의
model = DNN().to(device)

In [12]:
import torch.optim as optim

#Optimizer 정의
optimizer = optim.Adam(model.parameters(),lr=1e-3)

#Loss 정의 (다중분류문제이므로 CrossEntropyLoss)
loss = nn.CrossEntropyLoss()

In [13]:
#모델 학습
epochs = 10000
for epoch in range(epochs + 1):
    
    optimizer.zero_grad()
    
    hypothesis = model(X_train)
    
    cost = loss(hypothesis, y_train)
    
    cost.backward()
    
    optimizer.step()
    
    if epoch % 1000 == 0:
        print(epoch, cost.item())

0 3.9990131855010986
1000 3.1346906325779855e-05
2000 4.299573447497096e-06
3000 6.649948431913799e-07
4000 4.1773418502089044e-07
5000 1.335604959962211e-08
6000 0.0138386320322752
7000 3.48419448847892e-09
8000 9.553539257467492e-07
9000 0.0002197050635004416
10000 3.650662620202638e-05


In [14]:
#학습된 모델을 이용해서 Test 데이터 예측
with torch.no_grad():
    model.eval()
    
    hypothesis = model(X_test)
    predict = torch.argmax(hypothesis, dim=1)

In [15]:
#예측한 값을 csv로 반환
sample['label'] = predict.cpu().detach().numpy()
sample.to_csv('submit.csv',index=False)